# Library

In [18]:
#Library
import sys
import os
import os.path as pth

#!pip install torchplot
import torch
import torch.nn as nn
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init

import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets

import torch.backends.cudnn as cudnn
import torchplot as plt



import nets
import datasets
import tools
import layers as L
import train

from io import BytesIO
from datetime import datetime
from pytz import timezone
from slacker import Slacker
from quantization import *


import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
import random
import itertools
import math

from tqdm import tqdm
from numba import jit

import warnings
warnings.simplefilter("ignore")

# Functions

In [19]:
# Custom functions - Completed

@jit
def amac(x, w):
    return (x * w).sum()


@jit
def relu(x):
    return np.maximum(0,x)


@jit
def padding(a):
    result = np.zeros((a.shape[0],a.shape[1]+2,a.shape[2]+2))
    for i in range(a.shape[0]):
        result[i] = np.pad(a[i],1)
    return result


@jit(nopython=True)
def maxpooling(x_original):
   
    depth = x_original.shape[0] 
    row = int((x_original.shape[1])/2)
    col = int((x_original.shape[2])/2)

    one_layer = np.zeros((depth,row,col))
    
    for d in range(depth):
        for i in range(row):
            for j in range(col):
                r = x_original[d,2*i:2*i+2,2*j:2*j+2].max()
                one_layer[d,i,j] = r
    return one_layer


@jit(nopython=True)
def quant_signed_05_np(original, bit=5):
    bit = bit -1
    original = np.clip(original, -0.9375, 0.9375)
    original = original * (2**bit)
    
    (row, col) = original.shape
    result = np.zeros((row,col))
    for i in range(row):
        for j in range(col):
            result[i,j] = math.trunc(original[i,j])/ (2**bit)
    return result

@jit
def quant_signed_05_np_fc(original, bit=5):
    bit = bit -1
    original = np.clip(original, -0.9375, 0.9375)
    original = original * (2**bit)
    
    result = math.trunc(original)/ (2**bit)
    return result


get_bin = lambda x, n: format(x, 'b').zfill(n).replace("-","1")

# Partial sum function with point parameters - Completed

# This is partial sum function for convolution layer -> Matches with verilog
@jit(cache=True)
def partial_sum_fa_conv_point(original, bit=5, point = 1):
    a = original
    bit = bit - 2  + (point)
    value = 1.875/(2**(point))
    
    result = np.zeros(a.shape[0])
    for d in range(a.shape[0]):
        partial = a[d].sum()
        if partial > value:
            partial = value
        elif partial < -value:
            partial = -value
        else:
            partial = partial
        result[d] = math.trunc(partial* (2**bit)) / (2**bit)
    return result.sum()

# This function matches with our verilog model & reasonable accuracy
@jit(cache=True)
def partial_sum_fa_fc_point(original, bit=5, point=1):
    a = original
    bit = bit - 2  + (point)
    value = 1.875/(2**(point))
    
    partial = a.sum()
    if partial > value:
        partial = value
    elif partial < -value:
        partial = -value
    else:
        partial = partial
    result= math.trunc(partial* (2**bit)) / (2**bit)
    return result

# FC & Conv function with point parameters - completed
# These functions have same fixed point with bias & match with verilog file
@jit(cache=True)
def fc_fa_05(x_original, w, b, p=1):
   
    filt = w.shape[0]
    stage = int(x_original.shape[1]/8)
    c = np.zeros((1,filt))
    for f in range(filt):
        re = 0
        for i in range(stage):
            r = x_original[0,i*8:i*8+8] * w[f,i*8:i*8+8]
            re = re + partial_sum_fa_fc_point(r, point=p)
        c[0,f] = quant_signed_05_np_fc(re + b[f])
    return c

@jit(cache=True)
def conv_custom_fa_05(x_original, w, b, p=1):
   
    filt = w.shape[0]
    depth = x_original.shape[0] 
    row = x_original.shape[1] - 2
    col = x_original.shape[2] - 2

    c = np.zeros((filt,row,col))
    one_layer = np.zeros((row,col))
    
    for f in range(filt):
        for i in range(row):
            for j in range(col):
                r = x_original[:,i:i+3,j:j+3] * w[f]
                one_layer[i,j] = partial_sum_fa_conv_point(r, point=p)
        c[f,:,:] = quant_signed_05_np(one_layer + b[f])
    return c

In [72]:
# Function of saving parameter
def save_conv_weight(weight, bias, file_name, path,include_bias = 1):
    if(type(path) != str):
        path = str(path)
    if not os.path.exists(path):
        os.makedirs(path)
        print(f"Directory {path} created.")
    # else:
    #     print(f"Directory {path} already exists.")
    if(type(file_name) != str):
        path = str(file_name)
    if ".txt" not in file_name:
        weight_name = file_name + "_weight.txt"
        bias_name = file_name + "_bias.txt"
    file = open(path + "/" + weight_name,'w')

    a = weight
    a = a * 16
    for d in range(a.shape[1]):
        for f in range(a.shape[0]):
            for i in range(3):
                for j in range(3):
                    file.write(get_bin(int(a[f,d,i,j]),5)+"\n")
    file.close()

    file = open(path + "/" + bias_name,'w')
    a = bias
    a = a * 16
    for i in range(a.shape[0]):
        if include_bias:
            file.write(get_bin(int(a[i]),5)+"\n")
        else:
            file.write("0\n")
    file.close()
    print(f"Save {file_name} weight to {path}/{weight_name}\nSave {file_name} bias   to {path}/{bias_name}")

def save_fc_weight(weight, bias, file_name, path, include_bias = 1):
    if(type(path) != str):
        path = str(path)
    if not os.path.exists(path):
        os.makedirs(path)
        print(f"Directory {path} created.")
    # else:
    #     print(f"Directory {path} already exists.")
    if(type(file_name) != str):
        path = str(file_name)
    if ".txt" not in file_name:
        file_name = file_name + ".txt"
    
    file = open(path + "/" + file_name,'w')

    weight = weight*16
    bias = bias * 16
    for w in range(0, weight.shape[0], 32):
        for mac in range(min(weight.shape[0] - w, 32)):
            if include_bias:
                file.write(get_bin(int(bias[w+mac]),5)+"\n")
            else:
                file.write("0\n")
        for z in range(0, weight.shape[1], 8):
            for mac in range(min(weight.shape[0] - w, 32)):
                for port in range(8):
                    file.write(get_bin(int(weight[w+mac,z+port]),5)+"\n")
                    
    file.close()
    print(f"Save {file_name[:-4]} weight to {path}/{file_name}")

def save_conv_result(x, file_name, path):
    if(type(path) != str):
        path = str(path)
    if not os.path.exists(path):
        os.makedirs(path)
        print(f"Directory {path} created.")
    # else:
    #     print(f"Directory {path} already exists.")
    if(type(file_name) != str):
        path = str(file_name)
    if ".txt" not in file_name:
        output_name = file_name + "_output.txt"
    file = open(path + "/" + output_name,'w')

    save = x*16
    for d in range(x.shape[0]):
        for i in range(x.shape[1]):
            for j in range(x.shape[2]):
                file.write(get_bin(int(save[d,i,j]),10)+"\n")
    file.close()

# Open model and file

In [85]:
# Load pretrained model and data

file = open('latest.txt' , 'r' )
line = file.readline()
pretrained_checkpoint = line
file.close()

# Default settings for arch, dataset, and checkpoint
arch = "CNN_627_large"
dataset = "cifar10"
batch_size = 256
pretrained_checkpoint = pretrained_checkpoint

# trainloader, _, testloader = datasets.get_mnist(batch_size)
model = nets.mnist_quant()

pretrained_ckpt = torch.load(pretrained_checkpoint)
model.load_state_dict(pretrained_ckpt['state_dict'])
print("########## Loaded checkpoint '{}'".format(pretrained_checkpoint))

#print("mnist dataset with different fixed point")
#with open("./data_quantized/quant_test_data_mnist.pkl","rb") as f:
#    data_list = pickle.load(f)
#with open("./data_quantized/quant_test_label_mnist.pkl","rb") as g:
#    label_list = pickle.load(g)
    

print("mnist dataset with same fixed point")
with open("./data_quantized/quant_test_data_mnist_05.pkl","rb") as f:
    data_list = pickle.load(f)
with open("./data_quantized/quant_test_label_mnist_05.pkl","rb") as g:
    label_list = pickle.load(g)
    
# Load Weights and biases

w1, w2, w3, w4, w5, w6 = model.conv1.weight, model.conv2.weight, model.conv3.weight, model.conv4.weight, model.fc5.weight, model.fc6.weight
b1, b2, b3, b4, b5, b6 = model.conv1.bias, model.conv2.bias, model.conv3.bias, model.conv4.bias, model.fc5.bias, model.fc6.bias

w1 = quant_signed_15(w1)
w2 = quant_signed_15(w2)
w3 = quant_signed_15(w3)
w4 = quant_signed_15(w4)
w5 = quant_signed_15(w5)
w6 = quant_signed_15(w6)


w1 = w1.data.numpy()
b1 = b1.data.numpy()

w2 = w2.data.numpy()
b2 = b2.data.numpy()

w3 = w3.data.numpy()
b3 = b3.data.numpy()

w4 = w4.data.numpy()
b4 = b4.data.numpy()

w5 = w5.data.numpy()
b5 = b5.data.numpy()

w6 = w6.data.numpy()
b6 = b6.data.numpy()

Quant MNIST
########## Loaded checkpoint './checkpoints_quant/mnist_quant_mnist/2_18_Time_16_36/checkpoint_7_98.3.tar'
mnist dataset with same fixed point


In [86]:
path = "weight_new_wo_bias"

save_conv_weight(w1, b1, "conv1", path,include_bias=0)
save_conv_weight(w2, b2, "conv2", path,include_bias=0)
save_conv_weight(w3, b3, "conv3", path,include_bias=0)
save_conv_weight(w4, b4, "conv4", path,include_bias=0)
save_fc_weight(w5, b5, "fc5", path,include_bias=0)
save_fc_weight(w6, b6, "fc6", path,include_bias=0)

Save conv1 weight to weight_new_wo_bias/conv1_weight.txt
Save conv1 bias   to weight_new_wo_bias/conv1_bias.txt
Save conv2 weight to weight_new_wo_bias/conv2_weight.txt
Save conv2 bias   to weight_new_wo_bias/conv2_bias.txt
Save conv3 weight to weight_new_wo_bias/conv3_weight.txt
Save conv3 bias   to weight_new_wo_bias/conv3_bias.txt
Save conv4 weight to weight_new_wo_bias/conv4_weight.txt
Save conv4 bias   to weight_new_wo_bias/conv4_bias.txt
Save fc5 weight to weight_new_wo_bias/fc5.txt
Save fc6 weight to weight_new_wo_bias/fc6.txt


In [87]:
path = "weight_new_w_bias"

save_conv_weight(w1, b1, "conv1", path)
save_conv_weight(w2, b2, "conv2", path)
save_conv_weight(w3, b3, "conv3", path)
save_conv_weight(w4, b4, "conv4", path)
save_fc_weight(w5, b5, "fc5", path)
save_fc_weight(w6, b6, "fc6", path)

Save conv1 weight to weight_new_w_bias/conv1_weight.txt
Save conv1 bias   to weight_new_w_bias/conv1_bias.txt
Save conv2 weight to weight_new_w_bias/conv2_weight.txt
Save conv2 bias   to weight_new_w_bias/conv2_bias.txt
Save conv3 weight to weight_new_w_bias/conv3_weight.txt
Save conv3 bias   to weight_new_w_bias/conv3_bias.txt
Save conv4 weight to weight_new_w_bias/conv4_weight.txt
Save conv4 bias   to weight_new_w_bias/conv4_bias.txt
Save fc5 weight to weight_new_w_bias/fc5.txt
Save fc6 weight to weight_new_w_bias/fc6.txt


In [81]:
w1, w2, w3, w4, w5, w6 = model.conv1.weight, model.conv2.weight, model.conv3.weight, model.conv4.weight, model.fc5.weight, model.fc6.weight
b1, b2, b3, b4, b5, b6 = model.conv1.bias, model.conv2.bias, model.conv3.bias, model.conv4.bias, model.fc5.bias, model.fc6.bias

# original
- bias   : _ . _ _ _ _

- input  : _ _ . _ _ _
- wegith : _ . _ _ _ _
- output : _ | _ _ . _ _ _  | _ _ _ _
- output : _ _ . _ _ _
 
- conv   : _ _ . _ _ _
- wegith : _ . _ _ _ _


# solution (problem)
- bias   : _ . _ _ _ _
- wegith : _ . _ _ _ _
- input  : _ _ . _ _ _
- output : _ _ | _ . _ _ _ _ | _ _ _

- conv   : _ . _ _ _ _

# solution2 
- bias   : _ . _ _ _ _
- wegith : _ _ . _ _ _
- input  : _ . _ _ _ _
- output : _ _ | _ . _ _ _ _ | _ _ _

- conv   : _ . _ _ _ _


# potential solution
- bias   : _ . _ _ _ _
- wegith : _ . _ _ _ _
- input  : _ . _ _ _ _
- output : _ | _ . _ _ _ _ | _ _ _ _

- conv   : _ . _ _ _ _


# Comparing result

In [23]:
file = open('./result_new_wo_bias/conv1_input.txt','r')
file2 = open('./data_extracted/conv1_input_0317.txt','r')
for i,j in zip(file,file2):
    if i.replace('\n','') != j.replace('\n',''):
        print(i.replace('\n',''), j.replace('\n',''))

In [24]:
file = open('./result_new_wo_bias/conv1_output.txt','r')
file2 = open('./data_extracted/conv1_output_0317.txt','r')
for i,j in zip(file,file2):
    if i.replace('\n','') != j.replace('\n',''):
        print(i.replace('\n',''), j.replace('\n',''))

1000000001 0000000000
0000000001 0000000000
0000000101 0000000010
0000000101 0000000010
0000000101 0000000010
0000000101 0000000010
0000000101 0000000010
0000000101 0000000010
0000000101 0000000010
0000000101 0000000010
0000000101 0000000010
0000000101 0000000010
0000000101 0000000010
0000000101 0000000010
0000000101 0000000010
0000000101 0000000010
0000000101 0000000010
0000000101 0000000010
0000000101 0000000010
0000000101 0000000010
0000000101 0000000010
0000000101 0000000010
0000000101 0000000010
0000000101 0000000010
0000000101 0000000010
0000000101 0000000010
0000000101 0000000010
0000000101 0000000010
0000000110 0000000011
0000000100 0000000010
1000000101 1000000010
1000000101 1000000010
1000001001 1000000100
1000001001 1000000100
1000001001 1000000100
1000001001 1000000100
1000001001 1000000100
1000001001 1000000100
1000001001 1000000100
1000001001 1000000100
1000001001 1000000100
1000001001 1000000100
1000001001 1000000100
1000001001 1000000100
1000001001 1000000100
1000001001

1000000010 1000000001
1000000010 1000000001
0000000010 0000000001
0000000101 0000000010
0000000100 0000000010
1000000001 0000000000
1000000010 1000000001
1000000010 1000000001
1000000010 1000000001
1000000010 1000000001
1000000010 1000000001
1000000010 1000000001
1000000010 1000000001
1000000010 1000000001
1000000010 1000000001
1000000010 1000000001
1000000010 1000000001
0000000010 0000000001
0000000001 0000000000
1000000100 1000000010
1000000011 1000000001
1000000010 1000000001
1000000010 1000000001
1000000010 1000000001
1000000010 1000000001
1000000010 1000000001
1000000010 1000000001
1000000010 1000000001
1000000010 1000000001
0000000001 0000000000
0000000100 0000000010
0000000111 0000000011
0000000100 0000000010
1000001000 1000000100
1000000011 1000000001
1000000010 1000000001
1000000010 1000000001
1000000010 1000000001
1000000010 1000000001
1000000010 1000000001
1000000010 1000000001
1000000010 1000000001
1000000010 1000000001
1000000010 1000000001
1000000010 1000000001
1000000010

1000001111 1000001001
1000001111 1000001001
1000001000 1000000100
1000001000 1000000100
1000001000 1000000100
1000001000 1000000100
1000000011 1000000001
1000000010 1000000001
0000000010 0000000001
0000000111 0000000011
0000001010 0000000101
0000000011 0000000001
0000000111 0000000011
0000001000 0000000100
0000000101 0000000010
0000000101 0000000010
0000000111 0000000011
0000001111 0000001100
0000001111 0000001001
1000001110 1000000111
1000001111 1000001101
1000001110 1000000111
1000001000 1000000100
1000001000 1000000100
1000001000 1000000100
1000001000 1000000100
0000001011 0000000101
0000001010 0000000101
1000001111 1000001001
1000001111 1000001001
1000001000 1000000100
1000001000 1000000100
1000001000 1000000100
1000001000 1000000100
1000001000 1000000100
1000001000 1000000100
1000001000 1000000100
1000001000 1000000100
1000001000 1000000100
1000000110 1000000011
1000000011 1000000001
1000000110 1000000011
1000000100 1000000010
1000000010 1000000001
1000000011 1000000001
1000000010

1000001111 1000001001
1000001000 1000000100
1000001000 1000000100
1000001000 1000000100
1000001000 1000000100
1000001000 1000000100
1000001000 1000000100
1000001000 1000000100
1000001000 1000000100
0000000001 0000000000
0000001111 0000001010
0000001111 0000001000
0000000110 0000000011
1000001111 1000001001
1000001111 1000001101
1000001100 1000000110
1000001000 1000000100
1000001000 1000000100
1000001000 1000000100
1000001000 1000000100
1000001000 1000000100
1000001000 1000000100
1000001000 1000000100
1000001000 1000000100
1000001000 1000000100
1000001000 1000000100
1000001000 1000000100
0000001011 0000000101
0000001010 0000000101
1000001111 1000001001
1000001111 1000001001
1000001000 1000000100
1000001000 1000000100
1000001000 1000000100
1000001000 1000000100
1000001000 1000000100
1000001000 1000000100
1000001000 1000000100
1000001000 1000000100
0000000111 0000000011
0000001111 0000001100
0000001110 0000000111
1000000100 1000000010
1000001111 1000001010
1000001111 1000001001
1000001010

1000000001 0000000000
1000000001 0000000000
1000000001 0000000000
1000000001 0000000000
0000000110 0000000011
1000000001 0000000000
1000000111 1000000011
1000000001 0000000000
1000000001 0000000000
1000000001 0000000000
1000000001 0000000000
1000001111 1000000111
1000001111 1000001010
1000001111 1000001011
1000001111 1000001110
1000001111 1000001110
1000001111 1000001110
1000001111 1000001110
1000001111 1000001110
1000001111 1000001110
1000001111 1000001110
1000001111 1000001101
1000001111 1000001100
1000001111 1000001000
1000000110 1000000011
1000000001 0000000000
1000000001 0000000000
1000000001 0000000000
1000000001 0000000000
0000000110 0000000011
1000000001 0000000000
1000000111 1000000011
1000000001 0000000000
1000000001 0000000000
1000000001 0000000000
1000000001 0000000000
0000000101 0000000010
0000001111 0000001000
0000001111 0000001000
0000001111 0000000111
0000001111 0000001000
0000001000 0000000100
1000000001 0000000000
1000001100 1000000110
1000001110 1000000111
1000001110

...

0000001111 0000001001
1000001111 1000001000
1000001111 1000001000
1000001111 1000000111
1000001111 1000000111
1000001111 1000000111
1000001111 1000000111
1000000100 1000000010
1000000001 0000000000
1000001011 1000000101
1000001110 1000000111
1000001111 1000000111
1000001111 1000000111
1000001111 1000000111
1000001111 1000000111
1000001111 1000001001
1000001111 1000001000
1000001000 1000000100
1000001010 1000000101
1000001100 1000000110
1000001001 1000000100
1000000010 1000000001
0000000011 0000000001
0000000011 0000000001
0000000010 0000000001
0000001001 0000000100
0000001111 0000001011
0000001111 0000001000
1000000011 1000000001
1000000100 1000000010
1000001010 1000000101
1000001111 1000000111
1000001111 1000000111
1000001111 1000000111
1000001111 1000000111
1000000100 1000000010
1000000001 0000000000
1000001011 1000000101
1000001110 1000000111
1000001111 1000000111
1000001111 1000000111
1000001111 1000000111
1000001111 1000000111
1000001111 1000000111
1000001111 1000000111
1000001111

1000001111 1000000111
1000000100 1000000010
1000000001 0000000000
1000001011 1000000101
1000001110 1000000111
1000001111 1000000111
1000001111 1000000111
1000001111 1000000111
1000001111 1000000111
1000001111 1000000111
1000001111 1000000111
1000001111 1000000111
1000001111 1000000111
1000000001 0000000000
0000000101 0000000010
0000000100 0000000010
1000000011 1000000001
0000000011 0000000001
1000000001 0000000000
1000001100 1000000110
1000001111 1000000111
1000001111 1000000111
1000001111 1000000111
1000001111 1000000111
1000001111 1000000111
1000001111 1000000111
1000001111 1000000111
1000001111 1000000111
1000001111 1000000111
1000001111 1000000111
1000001111 1000000111
1000000100 1000000010
1000000001 0000000000
1000001011 1000000101
1000001110 1000000111
1000001111 1000000111
1000001111 1000000111
1000001111 1000000111
1000001111 1000000111
1000001111 1000000111
1000001111 1000000111
1000001111 1000000111
1000001111 1000000111
1000001111 1000000111
1000000101 1000000010
0000000110

1000000110 1000000011
1000000110 1000000011
1000000110 1000000011
1000000110 1000000011
1000000110 1000000011
1000000110 1000000011
1000000110 1000000011
1000000110 1000000011
1000000110 1000000011
1000000110 1000000011
1000000110 1000000011
1000000110 1000000011
1000001011 1000000101
1000001111 1000001110
1000001111 1000001011
0000001111 0000001000
0000001111 0000001110
0000001000 0000000100
1000000101 1000000010
1000000110 1000000011
1000000110 1000000011
1000000110 1000000011
1000000110 1000000011
1000000110 1000000011
1000001111 1000001010
1000001000 1000000100
0000001110 0000000111
0000000010 0000000001
1000000110 1000000011
1000000110 1000000011
1000000110 1000000011
1000000110 1000000011
1000000110 1000000011
1000000110 1000000011
1000000110 1000000011
1000000110 1000000011
1000000110 1000000011
1000000110 1000000011
1000000110 1000000011
1000000110 1000000011
1000000110 1000000011
1000000110 1000000011
1000001111 1000001000
1000001111 1000001110
1000000101 1000000010
0000001111

1000001111 1000001001
1000001111 1000001001
1000001111 1000001000
1000001011 1000000101
1000000010 1000000001
1000000111 1000000011
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001000
1000001011 1000000101
1000000010 1000000001
1000000111 1000000011
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111

1000001011 1000000101
1000000010 1000000001
1000000111 1000000011
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001001 1000000100
0000001001 0000000100
0000001100 0000000110
1000000101 1000000010
1000001111 1000001000
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001000
1000001011 1000000101
1000000010 1000000001
1000000111 1000000011
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111 1000001001
1000001111

In [ ]:
file = open('./result_new_wo_bias/pool_relu1_output.txt','r')
file2 = open('./data_extracted/pool_relu1_output_0317.txt','r')
for i,j in zip(file,file2):
    print(i.replace('\n',''), j.replace('\n',''))
    #if i.replace('\n','') != j.replace('\n',''):
        #print(i.replace('\n',''), j.replace('\n',''))

In [59]:
file = open('./result_new_wo_bias/conv2_output.txt','r')
file2 = open('./data_extracted/conv2_output_0317.txt','r')
for i,j in zip(file,file2):
    # print(i.replace('\n',''), j.replace('\n',''))
    if i.replace('\n','') != j.replace('\n',''):
        print(i.replace('\n',''), j.replace('\n',''))

1000000001 1000000010
1000000001 1000000010
1000000001 1000000010
1000000001 1000000010
1000000001 1000000010
1000000001 1000000010
1000000001 1000000010
1000000001 1000000010
1000000001 1000000010
1000000001 1000000010
1000000001 1000000010
1000000001 1000000010
1000000001 1000000010
0000000000 1000000001
1000000001 1000000010
1000000001 1000000010
1000000001 1000000010
1000000001 1000000010
1000000001 1000000010
1000000001 1000000010
1000000001 1000000010
1000000001 1000000010
1000000001 1000000010
1000000001 1000000010
1000000001 1000000010
1000000001 1000000010
0000000000 1000000001
1000000001 1000000010
0000000000 1000000001
0000000000 1000000001
0000000000 1000000001
0000000000 1000000001
0000000000 1000000001
0000000000 1000000001
0000000000 1000000010
0000000000 1000000010
0000000000 1000000010
1000000001 1000000010
1000000001 1000000010
1000000101 1000000111
1000000100 1000000110
1000000100 1000000110
1000000100 1000000101
1000000010 1000000100
1000000010 1000000011
1000000001

1000000010 1000000001
0000000000 1000000001
0000000000 1000000001
1000000010 1000000011
1000000010 1000000011
0000000000 0000000001
0000000000 0000000001
0000000000 0000000001
0000000000 0000000010
1000000001 0000000000
1000000001 0000000001
1000000010 0000000000
1000000011 1000000001
1000000011 1000000100
1000000001 1000000011
0000000001 0000000010
0000000000 0000000001
0000000000 1000000001
1000000010 1000000011
1000000100 1000000110
1000000101 1000001001
0000000000 0000000001
0000000000 0000000001
0000000000 1000000001
1000000100 1000000110
1000000011 1000000111
1000000100 1000000110
0000000000 0000000010
0000000000 0000000001
0000000000 1000000100
1000000011 1000000110
0000000000 0000000001
0000000000 0000000001
1000000001 1000000011
1000000011 1000000110
1000000010 1000000100
0000000000 0000000010
0000000000 0000000001
0000000000 1000000010
0000000000 1000000101
1000000001 1000000100
0000000000 0000000001
0000000000 0000000001
1000000010 1000000110
1000000011 1000000110
1000000001

1000000111 1000001001
1000000111 1000001110
1000000111 1000001111
1000000111 1000001111
1000000111 1000001111
1000000111 1000001111
1000000111 1000001010
1000000111 1000001010
1000000111 1000001010
1000000111 1000001110
1000000111 1000001111
1000000111 1000001111
1000000111 1000001111
1000000111 1000001111
1000000111 1000001111
1000000111 1000001101
1000000111 1000001000
1000000111 1000001010
1000000111 1000001111
1000000111 1000001111
1000000111 1000001111
1000000111 1000001110
1000000111 1000001111
1000000111 1000001111
1000000111 1000001111
1000000111 1000001111
1000000111 1000001111
1000000111 1000001111
1000000111 1000001111
1000000111 1000001111
1000000111 1000001100
1000000111 1000001000
1000000111 1000001000
1000000111 1000001101
1000000111 1000001111
1000000111 1000001110
1000000111 1000001000
0000000000 0000000001
0000000000 1000000001
0000000000 1000000001
0000000000 1000000001
0000000000 1000000001
0000000000 1000000001
0000000000 1000000001
0000000000 1000000001
0000000000

1000000001 0000000000
1000000001 0000000000
1000000001 0000000000
1000000001 0000000000
0000000000 0000000001
1000000010 1000000001
1000000001 1000000010
1000000001 1000000010
1000000001 1000000010
1000000010 1000000011
1000000010 1000000001
1000000010 1000000001
1000000001 1000000010
1000000001 1000000010
0000000100 0000000101
0000000111 0000001001
0000000010 0000000100
0000000000 0000000001
1000000011 1000000101
1000000010 1000000011
1000000001 1000000010
1000000101 1000000100
1000000100 1000000011
1000000001 1000000010
1000000001 1000000010
1000000100 1000000111
0000000111 0000001000
0000000111 0000001111
0000000111 0000001111
0000000111 0000001001
0000000111 0000001110
0000000101 0000001010
0000000111 0000001011
0000000010 0000000000
1000000010 1000000011
1000000001 1000000010
1000000001 1000000010
1000000011 1000000100
1000000101 1000001001
1000000100 1000001010
1000000001 1000000111
0000000000 1000000101
1000000011 1000001000
1000000111 1000001111
1000000010 0000000000
1000000010

1000000111 1000001111
1000000111 1000001111
1000000111 1000001111
1000000111 1000001100
1000000100 1000000101
1000000011 1000000100
1000000100 1000000110
1000000101 1000000110
1000000101 1000000110
1000000100 1000000110
1000000111 1000001100
1000000111 1000001111
1000000111 1000001111
1000000111 1000001111
1000000101 1000001001
1000000100 1000000101
1000000011 1000000100
1000000100 1000000110
1000000111 1000001111
1000000111 1000001111
1000000111 1000001111
1000000111 1000001111
1000000100 1000000110
1000000011 1000000100
1000000100 1000000110
1000000111 1000001001
1000000111 1000001111
1000000111 1000001111
1000000111 1000001111
1000000101 1000001010
1000000101 1000000110
1000000011 1000000100
1000000100 1000000110
1000000101 1000000110
1000000111 1000001111
1000000111 1000001111
1000000111 1000001111
1000000111 1000001110
1000000100 1000000110
1000000011 1000000100
1000000100 1000000110
1000000111 1000001110
1000000111 1000001111
1000000111 1000001111
1000000111 1000001111
1000000101

1000000111 1000001111
1000000111 1000001111
1000000111 1000001111
1000000111 1000001001
1000000001 1000000010
1000000010 1000000001
1000000010 1000000001
0000000100 0000000101
1000000010 1000000100
0000000000 1000000100
1000000111 1000001111
1000000011 1000001011
1000000111 1000001111
1000000110 1000001011
1000000110 1000000100
1000000100 1000000011
1000000001 1000000010
1000000010 1000000001
1000000010 1000000011
1000000001 1000000010
1000000101 1000000100
0000000010 0000000111
0000000011 0000000010
1000000001 0000000011
1000000001 1000000010
1000000010 1000000001
1000000111 1000001010
1000000101 1000000011
0000000010 0000000111
1000000001 0000000010
1000000011 1000000001
1000000001 1000000010
1000000010 1000000001
1000000111 1000001000
1000000111 1000001000
1000000001 0000000011
0000000111 0000001010
0000000000 0000000100
1000000001 1000000010
1000000010 1000000001
1000000111 1000001100
0000000000 0000000100
0000000000 0000000011
1000000011 1000000001
1000000001 1000000010
1000000010

1000000111 1000001110
1000000111 1000001110
1000000111 1000001110
1000000111 1000001110
1000000111 1000001110
1000000111 1000001110
1000000111 1000001110
1000000111 1000001110
1000000111 1000001110
1000000111 1000001110
1000000111 1000001110
1000000111 1000001111
1000000111 1000001101
1000000101 1000000110
0000000010 0000000000
0000000000 0000000001
1000000010 0000000000
1000000011 1000000010
1000000110 1000000111
1000000111 1000001000
1000000111 1000001000
1000000111 1000001000
0000000010 0000000000
0000000000 1000000011
1000000011 1000001101
1000000010 1000001001
1000000101 1000001010
1000000111 1000001110
1000000111 1000001011
1000000111 1000001100
1000000100 1000000110
1000000100 1000000110
1000000111 1000001000
0000000010 0000000000
1000000111 1000001100
1000000111 1000001111
1000000111 1000001111
1000000111 1000001111
1000000111 1000001111
1000000111 1000001111
1000000111 1000001110
1000000111 1000001001
1000000011 1000000110
1000000111 1000001010
0000000010 0000000000
1000000111

In [39]:
file = open('./result_new_wo_bias/conv3_output.txt','r')
file2 = open('./data_extracted/conv3_output_0317.txt','r')
num = 1
for i,j in zip(file,file2):
    print(num, i.replace('\n',''), j.replace('\n',''))
    num += 1
    #if i.replace('\n','') != j.replace('\n',''):
    #    print(i.replace('\n',''), j.replace('\n',''))

1 0000000000 0000000000
2 0000000000 0000000000
3 0000000000 0000000000
4 0000000000 0000000000
5 0000000000 0000000000
6 0000000000 0000000000
7 1000000001 0000000000
8 0000000000 0000000000
9 0000000001 0000000000
10 0000000000 0000000000
11 0000000001 0000000001
12 0000000000 1000000001
13 0000000001 0000000000
14 0000000001 0000000000
15 0000000000 0000000000
16 0000000000 0000000000
17 0000000001 0000000000
18 0000000000 1000000001
19 1000000010 1000000010
20 1000000010 1000000010
21 1000000010 1000000010
22 0000000001 0000000000
23 1000000010 0000000000
24 0000000000 0000000000
25 0000000000 0000000000
26 0000000000 0000000000
27 0000000000 0000000000
28 0000000000 0000000000
29 0000000001 0000000001
30 0000000001 0000000000
31 0000000000 0000000000
32 0000000000 0000000000
33 0000000001 0000000000
34 0000000000 0000000000
35 0000000000 0000000000
36 0000000000 0000000000
37 0000000000 1000000001
38 0000000000 1000000001
39 1000000001 1000000001
40 0000000000 0000000000
41 000000

505 1000000010 1000000001
506 1000000001 1000000001
507 1000000011 0000000001
508 1000001100 1000000111
509 1000001000 1000000101
510 1000000100 1000000001
511 1000000010 1000000001
512 1000000010 0000000000
513 1000000001 0000000000
514 0000000000 0000000001
515 0000000000 0000000001
516 0000000000 0000000000
517 0000000000 0000000000
518 0000000000 0000000000
519 1000000001 0000000000
520 1000000001 1000000001
521 1000000100 0000000000
522 1000000011 1000000010
523 1000000101 1000000001
524 0000000010 1000000001
525 0000000011 0000000100
526 0000000110 0000000101
527 0000000101 0000000011
528 0000000001 0000000000
529 1000000001 0000000001
530 1000000010 0000000000
531 1000000010 0000000001
532 1000000001 0000000001
533 1000000100 1000001000
534 1000000011 1000001000
535 1000000001 1000000110
536 0000000000 1000000001
537 0000000000 0000000000
538 0000000001 0000000000
539 0000000101 0000000011
540 1000000011 0000000001
541 1000001100 1000000011
542 1000000101 1000000101
543 00000000

884 1000001111 1000001111
885 1000001111 1000001110
886 1000000001 0000000001
887 1000001001 0000000000
888 1000000101 1000000010
889 1000000101 1000000001
890 1000000111 1000000100
891 1000001101 1000001010
892 1000000101 1000000110
893 1000000001 0000000000
894 1000000100 1000000011
895 1000000101 1000000010
896 1000000011 0000000000
897 1000000010 0000000000
898 1000000100 1000000001
899 1000000101 1000000001
900 1000000101 1000000001
901 1000000101 1000000001
902 1000000101 1000000010
903 1000000110 1000000010
904 1000000100 1000000011
905 0000000000 0000000000
906 1000001001 1000000110
907 1000001111 1000001100
908 1000001110 1000001100
909 1000001100 1000001000
910 1000001000 1000000101
911 0000000001 0000000000
912 0000000001 0000000000
913 0000000010 0000000000
914 1000000011 1000000001
915 1000000111 1000000111
916 1000000111 1000000110
917 1000001101 1000001010
918 1000001011 1000001010
919 1000000100 1000000001
920 0000000000 0000000000
921 1000000010 1000000001
922 10000000

1379 0000000000 0000000000
1380 0000000000 0000000000
1381 0000000001 0000000000
1382 0000000000 1000000010
1383 0000000000 0000000000
1384 1000000001 1000000001
1385 0000000010 0000000000
1386 1000000011 0000000000
1387 0000000000 0000000000
1388 0000000000 0000000000
1389 0000000000 0000000000
1390 0000000000 0000000000
1391 0000000000 0000000000
1392 0000000000 0000000000
1393 0000000010 0000000000
1394 1000000101 0000000000
1395 1000000001 0000000000
1396 1000000001 1000000001
1397 1000000001 1000000001
1398 0000000000 1000000001
1399 0000000000 0000000000
1400 0000000000 0000000000
1401 0000000001 0000000000
1402 1000000001 0000000000
1403 0000000000 0000000000
1404 0000000000 0000000000
1405 1000000001 0000000000
1406 0000000000 0000000000
1407 0000000000 0000000000
1408 0000000000 0000000000
1409 1000000001 0000000000
1410 1000000001 1000000001
1411 1000000010 1000000001
1412 1000000011 1000000010
1413 1000000011 1000000010
1414 1000000011 1000000010
1415 1000000100 1000000001
1

1933 0000000001 0000000000
1934 0000000011 0000000000
1935 1000000001 0000000001
1936 1000000001 0000000000
1937 0000000000 0000000000
1938 1000000011 1000000011
1939 1000000101 1000000100
1940 1000000100 1000000100
1941 1000000100 1000000010
1942 0000000010 0000000001
1943 1000000010 0000000000
1944 1000000001 0000000000
1945 0000000000 0000000000
1946 1000000001 0000000000
1947 1000000001 0000000000
1948 1000000001 0000000000
1949 1000000010 1000000001
1950 0000000000 0000000000
1951 1000000001 1000000001
1952 1000000001 0000000000
1953 0000000000 0000000000
1954 1000000001 0000000000
1955 0000000000 0000000000
1956 0000000000 0000000001
1957 0000000000 0000000000
1958 0000000000 1000000001
1959 1000000010 1000000001
1960 1000000001 0000000000
1961 0000000000 0000000000
1962 1000000010 0000000000
1963 1000000010 0000000000
1964 0000000010 0000000010
1965 1000000010 1000000001
1966 1000000010 1000000011
1967 1000000001 1000000001
1968 1000000001 0000000000
1969 0000000000 0000000000
1

...

6376 1000000110 1000000010
6377 0000000000 0000000000
6378 1000000001 0000000000
6379 1000000010 1000000001
6380 1000001001 1000001001
6381 1000000011 0000000000
6382 0000001011 0000001010
6383 1000000111 1000000010
6384 1000000011 1000000010
6385 0000000001 0000000000
6386 1000000110 1000000011
6387 1000001001 1000000111
6388 1000001011 1000000110
6389 0000000111 0000001101
6390 0000000010 0000000110
6391 1000001010 0000000000
6392 1000000011 0000000000
6393 0000000000 0000000000
6394 1000000011 1000000001
6395 1000000100 1000000010
6396 0000000111 0000000011
6397 0000000111 0000001010
6398 1000000110 1000000100
6399 1000000011 1000000001
6400 1000000010 0000000000
6401 1000000001 0000000000
6402 0000000000 0000000000
6403 0000000000 0000000000
6404 0000000000 1000000001
6405 0000000000 1000000001
6406 1000000001 0000000000
6407 1000000011 0000000000
6408 1000000010 0000000000
6409 1000000010 1000000010
6410 1000000100 1000000100
6411 1000001110 1000000110
6412 1000001101 1000001010
6

6784 1000000001 0000000000
6785 1000000010 0000000000
6786 1000000100 0000000000
6787 1000000100 1000000001
6788 1000000100 0000000000
6789 1000000100 0000000000
6790 1000000100 0000000000
6791 1000000101 1000000011
6792 1000000011 1000000001
6793 1000000010 0000000000
6794 1000001001 1000000011
6795 1000001100 1000000100
6796 1000001001 1000000101
6797 1000001001 1000000011
6798 1000000111 1000000001
6799 1000001010 1000000011
6800 1000000101 1000000010
6801 0000000000 0000000000
6802 1000001001 1000000010
6803 1000001110 1000001010
6804 1000001111 1000001100
6805 1000001111 1000001110
6806 1000001010 1000000111
6807 1000001000 1000000100
6808 1000000001 0000000000
6809 1000000001 0000000000
6810 1000001111 1000000110
6811 1000001111 1000001101
6812 1000001111 1000001111
6813 1000001111 1000001111
6814 1000000111 1000000101
6815 1000001011 1000000110
6816 1000000010 0000000000
6817 0000000000 0000000000
6818 1000000110 1000000001
6819 0000000001 0000000010
6820 1000000011 1000000001
6

7370 1000000010 0000000110
7371 1000000011 1000000010
7372 1000001110 1000001110
7373 1000001111 1000001111
7374 1000001111 1000001101
7375 1000001001 1000000111
7376 1000000110 1000000010
7377 0000000000 0000000001
7378 1000001100 1000001001
7379 1000001111 1000001111
7380 1000001111 1000001111
7381 1000001111 1000001111
7382 1000001111 1000001111
7383 1000001100 1000001010
7384 1000000101 1000000010
7385 1000000001 1000000001
7386 0000000001 0000000011
7387 0000001000 0000000101
7388 1000000001 0000000000
7389 0000000000 1000000010
7390 1000001000 1000000100
7391 1000001100 1000000110
7392 1000000101 0000000000
7393 1000000010 0000000000
7394 1000000111 1000000010
7395 0000000001 0000000000
7396 0000000010 1000000011
7397 1000000110 1000001000
7398 1000000010 1000000010
7399 1000000010 1000000001
7400 1000000101 1000000001
7401 1000000011 1000000001
7402 1000001000 1000000010
7403 1000000100 0000000000
7404 1000001111 1000001011
7405 1000001101 1000001100
7406 1000000100 1000000100
7

7765 0000000001 1000000001
7766 1000000011 1000000011
7767 0000000010 1000000011
7768 0000000110 0000000010
7769 1000000001 0000000000
7770 0000000000 0000000000
7771 1000000011 1000000011
7772 1000001011 1000001000
7773 1000001100 1000001011
7774 1000001011 1000001010
7775 0000000000 1000000011
7776 0000000011 0000000010
7777 0000000000 0000000000
7778 1000000010 1000000001
7779 1000001000 1000000101
7780 1000001001 1000001001
7781 1000001111 1000001111
7782 1000000110 1000001000
7783 0000000100 1000000011
7784 0000000000 0000000000
7785 1000000001 0000000000
7786 0000000100 0000000001
7787 0000000001 0000000000
7788 1000000001 1000000100
7789 1000000100 1000000110
7790 0000000110 0000000010
7791 0000000100 0000000001
7792 0000000010 0000000000
7793 1000000100 0000000000
7794 0000000010 0000000000
7795 0000001001 0000000110
7796 1000000010 1000000011
7797 0000000001 1000000011
7798 0000000011 0000000001
7799 0000000101 0000000000
7800 0000000001 0000000000
7801 1000000001 0000000000
7

8184 0000000001 0000000000
8185 1000000001 0000000000
8186 1000000010 1000000001
8187 1000000011 1000000001
8188 0000000000 0000000000
8189 0000000000 1000000001
8190 1000000001 0000000000
8191 0000000000 0000000000
8192 0000000000 0000000000


In [ ]:
file = open('./result_new_wo_bias/pool_relu1_output.txt','r')
file2 = open('./data_extracted/pool_relu1_output_0317.txt','r')
for i,j in zip(file,file2):
    print(i.replace('\n',''), j.replace('\n',''))
    #if i.replace('\n','') != j.replace('\n',''):
        #print(i.replace('\n',''), j.replace('\n',''))

# Saving Result without bias

In [88]:
file = open('./result_new_wo_bias/conv1_input.txt','w')
# x_original = torch.randn(1, 32,32)
x_original = data_list[0].view(1,32,32)
print((x_original == quant_signed_05(x_original)).sum())
x_original = quant_signed_05(x_original)

x = x_original
x = x*8
print(x.shape)


for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            file.write(get_bin(int(x[d,i,j]),5)+"\n")
file.close()

b1 = np.zeros(32)
x = x_original
x = padding(x)
x = conv_custom_fa_05(x,w1,b1, p=1)


file = open('./result_new_wo_bias/conv1_output.txt','w')

save = x*16
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            file.write(get_bin(int(save[d,i,j]),10)+"\n")
file.close()


x = relu(maxpooling(x))

file = open('./result_new_wo_bias/pool_relu1_output.txt','w')

save = x*16
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            #print(x[d,i,j])
            file.write(get_bin(int(save[d,i,j]),5)+"\n")
file.close()

print(x.shape)
b2 = np.zeros(64)
x = padding(x)
x = conv_custom_fa_05(x,w2,b2, p=1)

file = open('./result_new_wo_bias/conv2_output.txt','w')

save = x*16
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            file.write(get_bin(int(save[d,i,j]),10)+"\n")
file.close()

x = relu(maxpooling(x))

file = open('./result_new_wo_bias/pool_relu2_output.txt','w')

save = x*16
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            file.write(get_bin(int(save[d,i,j]),5)+"\n")
file.close()

print(x.shape)
b3 = np.zeros(128)
x = padding(x)
x = conv_custom_fa_05(x,w3,b3, p=1)

file = open('./result_new_wo_bias/conv3_output.txt','w')

save = x*16
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            file.write(get_bin(int(save[d,i,j]),10)+"\n")
file.close()

x = relu(maxpooling(x))

file = open('./result_new_wo_bias/pool_relu3_output.txt','w')

save = x*16
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            file.write(get_bin(int(save[d,i,j]),5)+"\n")
file.close()

print(x.shape)
b4 = np.zeros(256)
x = padding(x)
x = conv_custom_fa_05(x,w4,b4, p=1)

file = open('./result_new_wo_bias/conv4_output.txt','w')

save = x*16
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            file.write(get_bin(int(save[d,i,j]),10)+"\n")
file.close()

x = relu(maxpooling(x))

file = open('./result_new_wo_bias/pool_relu4_output.txt','w')

save = x*16
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            #print(x[d,i,j])
            file.write(get_bin(int(save[d,i,j]),5)+"\n")
file.close()

print(x.shape)
b5 = np.zeros(32)

x = fc_fa_05(x,w5,b5, p=1)

file = open('./result_new_wo_bias/fc5_output.txt','w')

save = x*16
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
            file.write(get_bin(int(save[d,i]),10)+"\n")
file.close()


x = relu(x)
file = open('./result_new_wo_bias/relu5_output.txt','w')

save = x*16
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
            file.write(get_bin(int(save[d,i]),5)+"\n")
file.close()

print(x.shape)
b6 = np.zeros(10)

x = fc_fa_05(x,w6,b6, p=1)

file = open('./result_new_wo_bias/fc6_output.txt','w')

save = x*16
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
            file.write(get_bin(int(save[d,i]),10)+"\n")
file.close()

x = relu(x)

file = open('./result_new_wo_bias/relu6_output.txt','w')

save = x*16
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
            file.write(get_bin(int(save[d,i]),5)+"\n")
file.close()
print(x.shape)

tensor(1024)
torch.Size([1, 32, 32])
(32, 16, 16)
(64, 8, 8)
(128, 4, 4)
(256, 2, 2)
(1, 32)
(1, 10)


# Saving result with bias

In [90]:
w1, w2, w3, w4, w5, w6 = model.conv1.weight, model.conv2.weight, model.conv3.weight, model.conv4.weight, model.fc5.weight, model.fc6.weight
b1, b2, b3, b4, b5, b6 = model.conv1.bias, model.conv2.bias, model.conv3.bias, model.conv4.bias, model.fc5.bias, model.fc6.bias

w1 = quant_signed_15(w1)
w2 = quant_signed_15(w2)
w3 = quant_signed_15(w3)
w4 = quant_signed_15(w4)
w5 = quant_signed_15(w5)
w6 = quant_signed_15(w6)

w1 = w1.data.numpy()
b1 = b1.data.numpy()

w2 = w2.data.numpy()
b2 = b2.data.numpy()

w3 = w3.data.numpy()
b3 = b3.data.numpy()

w4 = w4.data.numpy()
b4 = b4.data.numpy()

w5 = w5.data.numpy()
b5 = b5.data.numpy()

w6 = w6.data.numpy()
b6 = b6.data.numpy()



file = open('./result_new_w_bias/conv1_input.txt','w')
# x_original = torch.randn(1, 32,32)
x_original = data_list[0].view(1,32,32)
print((x_original == quant_signed_05(x_original)).sum())
x_original = quant_signed_05(x_original)

x = x_original
x = x*8
print(x.shape)


for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            file.write(get_bin(int(x[d,i,j]),5)+"\n")
file.close()

x = x_original
x = padding(x)
x = conv_custom_fa_05(x,w1,b1, p=1)


file = open('./result_new_w_bias/conv1_output.txt','w')

save = x*16
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            file.write(get_bin(int(save[d,i,j]),10)+"\n")
file.close()


x = relu(maxpooling(x))

file = open('./result_new_w_bias/pool_relu1_output.txt','w')

save = x*16
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            #print(x[d,i,j])
            file.write(get_bin(int(save[d,i,j]),5)+"\n")
file.close()

print(x.shape)
x = padding(x)
x = conv_custom_fa_05(x,w2,b2, p=1)

file = open('./result_new_w_bias/conv2_output.txt','w')

save = x*16
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            file.write(get_bin(int(save[d,i,j]),10)+"\n")
file.close()

x = relu(maxpooling(x))

file = open('./result_new_w_bias/pool_relu2_output.txt','w')

save = x*16
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            #print(x[d,i,j])
            file.write(get_bin(int(save[d,i,j]),5)+"\n")
file.close()

print(x.shape)
x = padding(x)
x = conv_custom_fa_05(x,w3,b3, p=1)

file = open('./result_new_w_bias/conv3_output.txt','w')

save = x*16
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            file.write(get_bin(int(save[d,i,j]),10)+"\n")
file.close()

x = relu(maxpooling(x))

file = open('./result_new_w_bias/pool_relu3_output.txt','w')

save = x*16
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            #print(x[d,i,j])
            file.write(get_bin(int(save[d,i,j]),5)+"\n")
file.close()

print(x.shape)
x = padding(x)
x = conv_custom_fa_05(x,w4,b4, p=1)

file = open('./result_new_w_bias/conv4_output.txt','w')

save = x*16
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            file.write(get_bin(int(save[d,i,j]),10)+"\n")
file.close()

x = relu(maxpooling(x))

file = open('./result_new_w_bias/pool_relu4_output.txt','w')

save = x*16
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
        for j in range(x.shape[2]):
            #print(x[d,i,j])
            file.write(get_bin(int(save[d,i,j]),5)+"\n")
file.close()

print(x.shape)

x = fc_fa_05(x,w5,b5, p=1)

file = open('./result_new_w_bias/fc5_output.txt','w')

save = x*16
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
            file.write(get_bin(int(save[d,i]),10)+"\n")
file.close()


x = relu(x)
file = open('./result_new_w_bias/relu5_output.txt','w')

save = x*16
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
            file.write(get_bin(int(save[d,i]),5)+"\n")
file.close()

print(x.shape)

x = fc_fa_05(x,w6,b6, p=1)

file = open('./result_new_w_bias/fc6_output.txt','w')

save = x*16
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
            file.write(get_bin(int(save[d,i]),10)+"\n")
file.close()

x = relu(x)

file = open('./result_new_w_bias/relu6_output.txt','w')

save = x*16
for d in range(x.shape[0]):
    for i in range(x.shape[1]):
            file.write(get_bin(int(save[d,i]),5)+"\n")
file.close()
print(x.shape)

tensor(1024)
torch.Size([1, 32, 32])
(32, 16, 16)
(64, 8, 8)
(128, 4, 4)
(256, 2, 2)
(1, 32)
(1, 10)


# Testing

In [93]:
# Evaluate with Custom model
correct = 0
#randlist = [i for i in range(50)]
#randlist = random.sample(range(0, 10000), 10)
randlist = [i for i in range(50)]
for i in randlist:
    y = label_list[i]
    x = data_list[i].view(1,32,32).numpy()

    x = padding(x)
    x = conv_custom_fa_05(x,w1,b1, p=1)
    x = maxpooling(x)
    x = relu(x)

    x = padding(x)
    x = conv_custom_fa_05(x,w2,b2, p=2)#3
    x = maxpooling(x)
    x = relu(x)

    x = padding(x)
    x = conv_custom_fa_05(x,w3,b3, p=2)#4
    x = maxpooling(x)
    x = relu(x)

    x = padding(x)
    x = conv_custom_fa_05(x,w4,b4, p=2)#4
    x = maxpooling(x)
    x = relu(x)

    x = x.reshape(1,1024)

    x = fc_fa_05(x,w5,b5, p=2)#4
    x = relu(x)

    x = fc_fa_05(x,w6,b6, p=1)#1


    if (np.argmax(x) == y.item()):
        correct += 1

print(correct/len(randlist))

0.08
